In [1]:
# import library 
import pandas as pd 
import pickle 
import os 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

### Put all the proccessing into a function

In [3]:
def read_dataframe(filename):
    "Write a function to read and preprocessing data"
    # Read the dataset
    df_taxi = pd.read_parquet(filename)

    # Adjust dropoff & pickup to pandas datetime 
    df_taxi['lpep_pickup_datetime'] = pd.to_datetime(df_taxi.lpep_pickup_datetime)
    df_taxi['lpep_dropoff_datetime'] = pd.to_datetime(df_taxi.lpep_dropoff_datetime)
    
    # Calculate the duration (drop_off -  pick_up)
    df_taxi['duration'] = df_taxi.lpep_dropoff_datetime - df_taxi.lpep_pickup_datetime
    
    # Adjust the duration in minutes for prediction 
    df_taxi['duration_minutes'] = df_taxi['duration'].dt.total_seconds() / 60

    # Since there are a lot of duration less than 1 minutes. We filter only duration between 1 minutes to 99% percentile
    df_taxi = df_taxi[(df_taxi['duration_minutes'] >= 1) & (df_taxi['duration_minutes'] <= 60)]

    # Feature Engineering 
    categorical_variables = ['PULocationID', 'DOLocationID']
    numerical_variables = ['trip_distance']

    # Convert it into "str"
    df_taxi[categorical_variables] = df_taxi[categorical_variables].astype(str)
    
    return df_taxi

In [5]:
df_train = read_dataframe('../00-Dataset/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../00-Dataset/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

### Create the training pipeline 

In [10]:
# Feature Engineering 
categorical_variables = ['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [12]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [14]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


10.499110705861218

### Try with Lasso Regression

In [74]:
# Train the model 
lr = Lasso(alpha= 0.0001) 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

10.47054920225702

### Try To Combine the input features 

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [18]:
# Feature Engineering 
categorical_variables = ['PU_DO']  #['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [20]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [22]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715208946364

In [24]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [26]:
import sklearn 
print(sklearn.__version__)

1.4.2
